In [2]:
import torch
import numpy as np
from torch import nn
from torch.nn import functional as F
import matplotlib.pyplot as plt
from advertorch.attacks import LinfPGDAttack, L2PGDAttack, DDNL2Attack
import ipdb

In [3]:
input_shape = (17, 17)

In [4]:
class SmallCNN(nn.Module):
    def __init__(self, activation=F.relu):
        super(SmallCNN, self).__init__()
        self.in_normalize = nn.BatchNorm2d(1)
        self.conv1 = nn.Conv2d(1, 512, 17, padding=8, padding_mode='circular')
        self.activation = activation
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc2 = nn.Linear(512, 2)
        
    def forward(self, x):
        x = self.in_normalize(x)
        x = self.activation(self.conv1(x))
        x = self.pool(x)
        x = x.view(-1, 512*1)
        x = self.fc2(x)
        return x
    
class FCNN(nn.Module):
    def __init__(self, activation=F.relu):
        super(FCNN, self).__init__()
        self.in_normalize = nn.BatchNorm2d(1)
        self.fc1 = nn.Linear(input_shape[0]*input_shape[1], 512)
        self.activation = activation
        self.fc2 = nn.Linear(512, 2)
        
    def forward(self, x):
        x = self.in_normalize(x)
        x = x.view(-1, input_shape[0]*input_shape[1])
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
im_1 = np.zeros(input_shape)+0.5
im_2 = im_1.copy()
im_1[7][7]+=0.5
im_2[7][7]-=0.5

In [6]:
np.stack([im_1, im_2]).shape

(2, 17, 17)

In [43]:
# im_1 = im_1.
data = np.stack([np.expand_dims(im_1, axis=0), np.expand_dims(im_2, axis=0)], axis=0)
y = np.array([1, 0])
batch_X, batch_y = torch.tensor(data).float().cuda(), torch.tensor(y).long().cuda()
criterion = nn.CrossEntropyLoss()
model_cnn = model = SmallCNN()
#model_fc = model = FCNN()
model = model.cuda()
LR = 0.01
opt = torch.optim.SGD(model.parameters(), lr = LR)


for idx in range(2000):
    total_loss = 0
    total_acc = 0
    output = model(batch_X)
    loss = criterion(output, batch_y)
    opt.zero_grad()
    loss.backward()
    opt.step()
    total_loss+=loss*batch_y.size(0)
    total_acc += (output.max(1)[1] == batch_y).sum().item()
    if (idx+1) % 800==0:
        opt.param_groups[0].update(lr=LR/10)
    if (idx+1)%100==0:
        print("Epoch: {}, Train Acc: {:.3f}, Loss : {:.3f}".format(idx+1, total_acc*100./y.shape[0], total_loss/y.shape[0]))

    

Epoch: 100, Train Acc: 100.000, Loss : 0.580
Epoch: 200, Train Acc: 100.000, Loss : 0.422
Epoch: 300, Train Acc: 100.000, Loss : 0.237
Epoch: 400, Train Acc: 100.000, Loss : 0.117
Epoch: 500, Train Acc: 100.000, Loss : 0.063
Epoch: 600, Train Acc: 100.000, Loss : 0.038
Epoch: 700, Train Acc: 100.000, Loss : 0.026
Epoch: 800, Train Acc: 100.000, Loss : 0.019
Epoch: 900, Train Acc: 100.000, Loss : 0.018
Epoch: 1000, Train Acc: 100.000, Loss : 0.018
Epoch: 1100, Train Acc: 100.000, Loss : 0.017
Epoch: 1200, Train Acc: 100.000, Loss : 0.017
Epoch: 1300, Train Acc: 100.000, Loss : 0.017
Epoch: 1400, Train Acc: 100.000, Loss : 0.016
Epoch: 1500, Train Acc: 100.000, Loss : 0.016
Epoch: 1600, Train Acc: 100.000, Loss : 0.015
Epoch: 1700, Train Acc: 100.000, Loss : 0.015
Epoch: 1800, Train Acc: 100.000, Loss : 0.015
Epoch: 1900, Train Acc: 100.000, Loss : 0.014
Epoch: 2000, Train Acc: 100.000, Loss : 0.014


In [44]:
model_cnn

SmallCNN(
  (in_normalize): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(1, 512, kernel_size=(17, 17), stride=(1, 1), padding=(8, 8), padding_mode=circular)
  (pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [45]:
model_fc

FCNN(
  (in_normalize): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=289, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [46]:
model_cnn = model_cnn.eval()
model_fc = model_fc.eval()

In [47]:
model_cnn = model_cnn.to('cuda')
model_fc = model_fc.to('cuda')

In [48]:
def test_pgd(X, y, adversary, batch_size):
    SPLITS = int(X.shape[0]/batch_size)
    total_acc = 0
    for idx in range(SPLITS):
        s1, s2 = idx*batch_size, min((idx+1)*batch_size, X.shape[0])
#         ipdb.set_trace()
        batch_X, batch_y = X[s1:s2,:,:,:], y[s1:s2]
        batch_X, batch_y = torch.tensor(batch_X).float().cuda(), torch.tensor(batch_y).long().cuda()
#         ipdb.set_trace()
        adv_untargeted = adversary.perturb(batch_X, batch_y)
        robust_out = model(adv_untargeted)
        total_acc += (robust_out.max(1)[1] == batch_y).sum().item()
        
#     print("Test Robust Acc: {:.3f}".format(total_acc*100./y.shape[0]))
    return total_acc*100./y.shape[0]

def test_pgd_ddn(X, y, adversary, batch_size):
    SPLITS = int(X.shape[0]/batch_size)
    total_acc = 0
    for idx in range(SPLITS):
        s1, s2 = idx*batch_size, min((idx+1)*batch_size, X.shape[0])
#         ipdb.set_trace()
        batch_X, batch_y = X[s1:s2,:,:,:], y[s1:s2]
        batch_X, batch_y = torch.tensor(batch_X).float().cuda(), torch.tensor(batch_y).long().cuda()
#         ipdb.set_trace()
        adv_untargeted = adversary.perturb(batch_X, batch_y)
        robust_out = model(adv_untargeted)
        total_acc += (robust_out.max(1)[1] == batch_y).sum().item()
        
#     print("Test Robust Acc: {:.3f}".format(total_acc*100./y.shape[0]))
    return total_acc*100./y.shape[0], adv_untargeted

def l2_norms_diff(x, y):
    x = x.reshape(x.shape[0], -1).cpu().numpy()
    y = y.reshape(y.shape[0], -1)
    norms = np.linalg.norm(x-y, ord=2, axis=1)
    return norms, norms.mean(), np.median(norms)

In [49]:
range_input = (data.max() - data.min())

adversary = DDNL2Attack(model_cnn, loss_fn=nn.CrossEntropyLoss(reduction="sum"),
                nb_iter=1000, clip_min=data.min(), clip_max=data.max(), quantize=False,
                targeted=False)
robust_acc, adv_data = test_pgd_ddn(data, y, adversary, 2)
# print("Epsilon : {}, Robust Accuracy: {}".format(eps, robust_acc))
norms, mean_norm, median_norm = l2_norms_diff(adv_data, data)
print("CNN Mean Norm: {}".format(mean_norm))

adversary = DDNL2Attack(model_fc, loss_fn=nn.CrossEntropyLoss(reduction="sum"),
                nb_iter=1000, clip_min=data.min(), clip_max=data.max(), quantize=False,
                targeted=False)
robust_acc, adv_data = test_pgd_ddn(data, y, adversary, 2)
# print("Epsilon : {}, Robust Accuracy: {}".format(eps, robust_acc))
norms, mean_norm, median_norm = l2_norms_diff(adv_data, data)
print("FC Mean Norm: {}".format(mean_norm))

C:\Users\anass\anaconda3\envs\handson-ml3\lib\site-packages\advertorch\attacks\decoupled_direction_norm.py:48: UserWarning: This Attack currently does not support a different loss function other than the default. Setting loss_fn manually is not effective.
  warnings.warn(
C:\Users\anass\anaconda3\envs\handson-ml3\lib\site-packages\torch\optim\lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


CNN Mean Norm: 0.02951764893844461
FC Mean Norm: 0.33991949597671944


In [50]:
torch.norm(adv_data.reshape(data.shape[0], -1), p=2, dim=1)

tensor([8.5187, 8.4697], device='cuda:0')

In [51]:
adv_data

tensor([[[[0.4629, 0.5096, 0.5055, 0.5053, 0.4953, 0.4883, 0.4678, 0.4957,
           0.4837, 0.5155, 0.5070, 0.4833, 0.4976, 0.5073, 0.5128, 0.4872,
           0.4845],
          [0.4761, 0.5206, 0.4733, 0.5313, 0.4764, 0.4872, 0.5343, 0.4868,
           0.4983, 0.4921, 0.4755, 0.4996, 0.4965, 0.5097, 0.5126, 0.4796,
           0.4958],
          [0.5225, 0.4791, 0.5052, 0.5186, 0.4905, 0.5142, 0.4825, 0.5039,
           0.4962, 0.4962, 0.5000, 0.4875, 0.4954, 0.4844, 0.5019, 0.4928,
           0.4995],
          [0.4906, 0.5179, 0.5028, 0.5052, 0.4964, 0.5150, 0.4940, 0.5051,
           0.4711, 0.4781, 0.5207, 0.4985, 0.5065, 0.4872, 0.5053, 0.5105,
           0.5025],
          [0.4964, 0.5102, 0.4996, 0.5080, 0.5143, 0.4986, 0.5002, 0.4983,
           0.4920, 0.5163, 0.5360, 0.5107, 0.5210, 0.5124, 0.5061, 0.4723,
           0.4947],
          [0.5084, 0.5256, 0.5020, 0.5261, 0.5062, 0.4835, 0.4961, 0.5413,
           0.4773, 0.5071, 0.5001, 0.5032, 0.4991, 0.5023, 0.5163, 0.5310,


In [52]:
data.shape

(2, 1, 17, 17)

In [53]:
range_input = (data.max() - data.min())

for eps in [0.001, 0.002, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05]:
    range_attack = range_input*eps
    step_attack = range_attack/50
    adversary = LinfPGDAttack(model_cnn, loss_fn=nn.CrossEntropyLoss(reduction="sum"), eps=range_attack,
                    nb_iter=100, eps_iter=step_attack, rand_init=True, clip_min=data.min(), clip_max=data.max(),
                    targeted=False)
    robust_acc = test_pgd(data, y, adversary, 2)
    print("Epsilon : {}, Robust Accuracy: {}".format(eps, robust_acc))

Epsilon : 0.001, Robust Accuracy: 100.0
Epsilon : 0.002, Robust Accuracy: 0.0
Epsilon : 0.005, Robust Accuracy: 0.0
Epsilon : 0.01, Robust Accuracy: 0.0
Epsilon : 0.02, Robust Accuracy: 0.0
Epsilon : 0.03, Robust Accuracy: 0.0
Epsilon : 0.04, Robust Accuracy: 0.0
Epsilon : 0.05, Robust Accuracy: 0.0


In [54]:
range_input = (data.max() - data.min())

for eps in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]:
    range_attack = range_input*eps
    step_attack = range_attack/50
    adversary = L2PGDAttack(model_cnn, loss_fn=nn.CrossEntropyLoss(reduction="sum"), eps=range_attack,
                    nb_iter=100, eps_iter=step_attack, rand_init=True, clip_min=data.min(), clip_max=data.max(),
                    targeted=False)
    robust_acc = test_pgd(data, y, adversary, 2)
    print("Epsilon : {}, Robust Accuracy: {}".format(eps, robust_acc))

Epsilon : 0.01, Robust Accuracy: 100.0
Epsilon : 0.02, Robust Accuracy: 100.0
Epsilon : 0.03, Robust Accuracy: 100.0
Epsilon : 0.04, Robust Accuracy: 0.0
Epsilon : 0.05, Robust Accuracy: 0.0
Epsilon : 0.06, Robust Accuracy: 0.0
Epsilon : 0.07, Robust Accuracy: 0.0
Epsilon : 0.08, Robust Accuracy: 0.0


CNN Model <br>
Epsilon : 0.001, Robust Accuracy: 100.0 <br>
Epsilon : 0.002, Robust Accuracy: 100.0 <br>
Epsilon : 0.005, Robust Accuracy: 0.0 <br>
Epsilon : 0.01, Robust Accuracy: 0.0 <br>
Epsilon : 0.02, Robust Accuracy: 0.0 <br>
Epsilon : 0.03, Robust Accuracy: 0.0 <br>
Epsilon : 0.04, Robust Accuracy: 0.0 <br>
Epsilon : 0.05, Robust Accuracy: 0.0 <br>

FC Model <br>
Epsilon : 0.001, Robust Accuracy: 100.0 <br>
Epsilon : 0.002, Robust Accuracy: 100.0 <br>
Epsilon : 0.005, Robust Accuracy: 100.0 <br>
Epsilon : 0.01, Robust Accuracy: 100.0 <br>
Epsilon : 0.02, Robust Accuracy: 100.0 <br>
Epsilon : 0.03, Robust Accuracy: 100.0 <br>
Epsilon : 0.04, Robust Accuracy: 100.0 <br>
Epsilon : 0.05, Robust Accuracy: 0.0 <br>

## PREVIOUS EXPERIMENTS DISCARDDD

FC Model <br>
Epsilon : 0.01, Robust Accuracy: 100.0 <br>
Epsilon : 0.02, Robust Accuracy: 100.0 <br>
Epsilon : 0.03, Robust Accuracy: 100.0 <br>
Epsilon : 0.04, Robust Accuracy: 100.0 <br>
Epsilon : 0.05, Robust Accuracy: 100.0 <br>
Epsilon : 0.06, Robust Accuracy: 50.0 <br>
Epsilon : 0.07, Robust Accuracy: 100.0 <br>
Epsilon : 0.08, Robust Accuracy: 100.0 <br>
Epsilon : 0.09, Robust Accuracy: 0.0 <br>
Epsilon : 0.1, Robust Accuracy: 100.0 <br>
<br>
CNN Model <br>
Epsilon : 0.01, Robust Accuracy: 0.0 <br>
Epsilon : 0.02, Robust Accuracy: 0.0 <br>
Epsilon : 0.03, Robust Accuracy: 0.0 <br>
Epsilon : 0.04, Robust Accuracy: 0.0 <br>
Epsilon : 0.05, Robust Accuracy: 0.0 <br>
Epsilon : 0.06, Robust Accuracy: 0.0 <br>
Epsilon : 0.07, Robust Accuracy: 0.0 <br>
Epsilon : 0.08, Robust Accuracy: 0.0 <br>
Epsilon : 0.09, Robust Accuracy: 0.0 <br>
Epsilon : 0.1, Robust Accuracy: 0.0 <br>

In [ ]:
range_input = np.linalg.norm(data.reshape((data.shape[0], -1)), ord=2, axis=1).mean()

for eps in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
    range_attack = range_input*eps
    step_attack = range_attack/10
    adversary = L2PGDAttack(model_cnn, loss_fn=nn.CrossEntropyLoss(reduction="sum"), eps=range_attack,
                    nb_iter=50, eps_iter=step_attack, rand_init=True, clip_min=data.min(), clip_max=data.max(),
                    targeted=False)
    robust_acc = test_pgd(data, y, adversary, 2)
    print("Epsilon : {}, Robust Accuracy: {}".format(eps, robust_acc))